Reading the initial dollar dataset

In [ ]:
import pandas as pd

file_path = "registery_dollar_1738332855.csv"
df = pd.read_csv(file_path)

Function to remove emojis, symbols and extras from text using regex

In [ ]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\u2600-\u26FF"          # Miscellaneous Symbols
                           u"\u2700-\u27BF"          # Dingbats
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text.replace("تومان", "").strip()

Function to convert miladi date to jalali date

In [ ]:
from persiantools.jdatetime import JalaliDate
from datetime import date

def convert_miladi_to_jalali(d):
    year, month, day = map(int, d.split("-"))
    return JalaliDate.to_jalali(year, month, day).strftime("%Y-%m-%d")

Running the above functions row by row on the dataset and categorizing the relevant indices and prices

In [ ]:
data = []

for row in df.itertuples(index=False):
    description = row[0].split('\n \n')
    year, month, day = map(int, row[2].split("-"))
    s = {"time-date-jl": convert_miladi_to_jalali(row[2]), "time-date": date(year,month,day), "time-time": row[3]}
    for cell in description:
        if ':' not in cell:
            break
        cell = remove_emojis(cell)
        cell = cell.split(':')
        s[cell[0].strip()] = cell[1].replace(',','').strip()

    data.append(s)

Converting the collected data into a dataframe and loading it into Excel

In [ ]:
result_df = pd.DataFrame(data)
result_df.to_excel("dollar.xlsx", index=False)